# Assignment 4: Merging

Related text: https://ledatascifi.github.io/ledatascifi-2021/content/03/05b_merging.html

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import seaborn as sns
# import statsmodels.api as sm

## PART 1

Insert cell(s) below this one as needed to finish this Part.

Load the following two datasets and answer these questions. Assume that the French data is the "left" dataset and the stock data is the "right" dataset. 

Q1-Q9:
1. How many observations are there in `ff` data?
1. How many observations are there in `crsp` data?
4. After an inner merge?
1. How many observations are there after a left merge?
2. After a right merge? 
3. After an outer merge? 
1. Why isn't the answer to Q4 and Q5 the same?
1. Is this a 1:1, 1:M, M:1, or M:M merge?

Remember: Specify `how`, `on`, `indicator=True`, and `validate` on each merge!

In [2]:
ff = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily',start=1980,end=2010)[0] # the [0] is because the imported obect is a dictionary, and key=0 is the dataframe
ff = ff.reset_index().rename(columns={"Mkt-RF":"mkt_excess", "Date":"date"})
crsp = pd.read_stata('https://github.com/LeDataSciFi/ledatascifi-2021/blob/main/data/3firm_ret_1990_2020.dta?raw=true')
crsp['ret'] = crsp['ret']*100 # convert to precentage to match FF's convention on scaling


In [3]:
ff
ff.describe() #shows count
ff.index #shows 
len(ff) #shows count
print('1) There are ' + str(len(ff)) + ' observations in ff dataset')

1) There are 7571 observations in ff dataset


In [4]:
crsp
crsp.index
len(crsp)
print('2) There are ' + str(len(crsp)) + ' observations in crsp dataset')

2) There are 30236 observations in crsp dataset


In [5]:
Merged_inner = pd.merge(ff,crsp,how='inner',on='date',indicator=True,validate='one_to_many')
Merged_inner
Merged_inner.describe()
Merged_inner.index
len(Merged_inner)
print('3) There are ' + str(len(Merged_inner)) + ' observations in merged dataset')

3) There are 20172 observations in merged dataset


In [6]:
Merged_left = pd.merge(ff,crsp, how='left',on='date',indicator=True,validate='one_to_many')
Merged_left
Merged_left.describe()
Merged_left.index
len(Merged_left)
print('4) There are ' + str(len(Merged_left)) + ' observations in merged dataset')

4) There are 22700 observations in merged dataset


In [7]:
Merged_right = pd.merge(ff,crsp, how='right',on='date',indicator=True,validate='one_to_many')
Merged_right
Merged_right.describe()
Merged_right.index
len(Merged_right)
print('5) There are ' + str(len(Merged_right)) + ' observations in merged dataset')

5) There are 30236 observations in merged dataset


In [8]:
Merged_outer = pd.merge(ff,crsp, how='outer',on='date',indicator=True,validate='one_to_many')
Merged_outer
Merged_outer.describe()
Merged_outer.index
len(Merged_outer)
print('6) There are ' + str(len(Merged_outer)) + ' observations in merged dataset')

6) There are 32764 observations in merged dataset


7)  Both are joined on the the date key (ff from 1980 to 2009, crsp from 1990 to 2019. The crsp dataset has a column of stocks so each row must account for an individual stock. 
    With ff, all values are recorded daily. When we merge on the left, we have null values for each firm from 1980-1990 and only need to take the daily values once (as opposed to each firm).
    The difference in observations equals the number of firms observed times the number of recorded trading days from 1980 to 1990



8) This is techincally a 1:M merge (crsp dataset repeats merge key (date) for each stock (4 times))
- Note: Running it as a M:M also works even though it is a 1:M merge 
    - Can see this after you get the same results by running: 
``` pd.merge(ff,crsp,validate='many_to_many') ```
         Versus 
``` pd.merge(ff,crsp,validate='one_to_many')```
    

## PART 2: Creating variables around a merge 

Insert cell(s) below this one as needed to finish this Part.

Q9: Suppose you're analyzing the stock returns in the CRSP dataset. Add the FF variables to it. Then add a variable dataset that equals "the variance of the excess return on the market for the year" for each firm year. (These sentences are enough to pick which "how" option to choose.)  `describe()` the crsp dataset after you add the new variables - but you only need to describe `'ret','mkt_excess', 'SMB', 'HML'` and the new variance variable.

Note: You don't need to "annualize" the variance because the resulting data is daily.

HINT: If you get a mean of 1.334374 for the variance variable, that is wrong.

In [9]:
Merged = pd.merge(ff,crsp,how='right',on='date',indicator=True,validate='one_to_many')


Merged['Years'] = Merged['date'].dt.year

#option 1
# opt_1 = Merged.groupby(['Years'])['mkt_excess'].var().reset_index()
# Merged['var'] = opt_1['mkt_excess'] 
#mean var is 1.334860


#option 2
opt_2 = Merged.groupby(['date','Years'])['mkt_excess'].mean().reset_index()
var_ann = opt_2.groupby(['Years'])['mkt_excess'].var().reset_index()
Merged['var'] = var_ann['mkt_excess']
#mean var is 1.338848 (I think this one makes more sense)




Merged[['ret','mkt_excess','SMB','HML','var']].describe()

,ret,mkt_excess,SMB,HML,var
count,30236.000000,20172.000000,20172.000000,20172.000000,20.000000
mean,0.074904,0.023724,0.006889,0.017028,1.338848
std,2.221544,1.156874,0.587440,0.642751,1.431382
min,-51.869160,-8.950000,-4.180000,-4.390000,0.232199
25%,-0.980813,-0.480000,-0.330000,-0.260000,0.446636
50%,0.000000,0.060000,0.020000,0.000000,0.933522
75%,1.074159,0.550000,0.340000,0.270000,1.653194
max,33.227849,11.350000,4.490000,4.830000,6.343994


In [10]:
#ff.iloc[2528:2580]

## PART 3: Adding a new variable via merge

Insert cell(s) below this one as needed to finish this Part.

Imagine you're analyzing our CCM dataset. (The code to download the CCM data is on 3.2.5 of the website.) Add a variable call "PatentStock" from [here](https://github.com/LeDataSciFi/ledatascifi-2021/blob/main/data/two_pat_vars.csv) to it. (The sentences are enough to pick which "how" option to choose in the merge.)

- Q10: `.describe()` the "l_a" and the "PatentStock" variables after the merge. 
- Q11: Tabulate the _merge variable. 

HINT: As you've seen several times before, pandas can download a csv file with just a URL. The "hitch" here is that I'm not giving you the URL. 

In [11]:
# these three are used to download the file
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

In [12]:
url = 'https://github.com/LeDataSciFi/ledatascifi-2021/blob/main/data/CCM_cleaned_for_class.zip?raw=true'

with urlopen(url) as request:
    data = BytesIO(request.read())

with ZipFile(data) as archive:
    with archive.open(archive.namelist()[0]) as stata:
        ccm = pd.read_stata(stata)
        
#ccm

In [13]:
url_2 = 'https://raw.githubusercontent.com/LeDataSciFi/ledatascifi-2021/main/data/two_pat_vars.csv'
Patent = pd.read_csv(url_2)
#Patent

In [14]:
Mer = pd.merge(ccm,Patent,how='left',indicator=True)
#Mer = pd.merge(ccm,Patent,how='left', on=['gvkey','fyear'],indicator=True)

Mer[['l_a','PatentStock']].describe()

,l_a,PatentStock
count,222978.000000,90523.000000
mean,5.226964,80.851256
std,2.408634,514.576731
min,-6.907755,0.000000
25%,3.476993,0.272491
50%,5.094452,2.722500
75%,6.864064,15.020925
max,15.142903,23890.555000


In [15]:
Mer['_merge'].value_counts()

both          172315
left_only      50686
right_only         0
Name: _merge, dtype: int64